## Selection statement with more than two selection blocks

Try to implement a workflow in which all the operators of the selection interface *(IF, ELSEIF, ELSE, ENDIF)* are adopted.

The following figure shows the graphical representation of a possible workflow.

<img src="../imgs/Selection_Interface.svg" alt="Selection_Interface" width="300">

The corresponding pseudo-code could be something like this:

    if flag1 == 1:
        Import a NetCDF file and subset it at the same time
    else 
        if flag2 == 1:
            Create a container and then a new datacube with random data and dimensions
        else
            Import a NetCDF file, then perform a subset operation over the datacube dimensions

<hr style="height:1px;border-top:1px solid #0000FF" />

As first step, let's connect to the Ophidia Server and import the necessary modules

In [ ]:
from esdm_pav_client import Workflow, Experiment, Task
import sys
from PyOphidia import cube,client
cube.Cube.setclient(read_env=True)

<hr style="height:1px;border-top:1px solid #0000FF" />

Let's try to build step-by-step the experiment associated to the workflow, by completing the missing arguments (```.....```).

**Some hints:**
   - the task ```ELSEIF``` has to be a child of ```IF``` or another ```ELSEIF```
   - the task ```ELSE``` could be a child of ```ELSEIF``` and must be child of ```IF```
   - the last task of any branch should be added as a dependency of the ```ENDIF``` task
   - the workflow has the following arguments:
       -  *if condition* ```$1```
       -  *elseif condition* ```$2```
       -  *NetCDF filename* ```$3```
       -  *variable* to be imported ```$4```
       -  *subset filter* (lat|lon|time) ```$5```
 

First of all, we need to define the global workflow information.

In [ ]:
e1 = Experiment(
    name= "Selection Interface",
    author= "CMCC",
    abstract= "Selection statement with more than two selection blocks",
    exec_mode= "sync",
    ncores="1"
)

<hr style="height:1px;border-top:1px solid #0000FF" />

**IF block**

Open a ```if``` selection block and start the first selection sub-block.

The boolean expression to be checked is the first argument of the workflow: if ```true``` (a *numerical non-zero value*), the tasks depending on this task are executed, whereas the tasks associated with the following selection sub-blocks are skipped; if ```false``` (a *zero value*), the tasks depending on this task are skipped, whereas the tasks associated with the following selection sub-block are executed.

In [ ]:
t1 = e1.newTask(name="IF block",
                type="control",
                operator='.....',
                arguments={'condition': '$1'})

<hr style="height:1px;border-top:1px solid #0000FF" />

If the first condition is ```true```, use the ```oph_importnc``` operator to import and subset a NetCDF file at the same time. **Note** use the workflow variables ```$N``` for the import operator arguments.

In [ ]:
t2 = e1.newTask(name="Import and subset",
                type="ophidia",
                operator='.....',
                arguments={'measure': '.....', 
                           'import_metadata': 'yes',
                           'imp_dim': 'time', 
                           'imp_concept_level': 'd',
                           'hierarchy': 'oph_base|oph_base|oph_time',
                           'subset_dims': 'lat|lon|time',
                           'subset_filter': '.....',
                           'subset_type': 'coord',
                           'time_filter': 'no',
                           'description': 'Max Temp imported and subsetted',
                           'input': '.....'},
                dependencies={...:''})

<hr style="height:1px;border-top:1px solid #0000FF" />

Start a new sub-block of the selection block ```if```. Here the boolean expression to be checked is the second argument of the workflow.

In [ ]:
t3 = e1.newTask(name="ELSEIF block",
                type="control",
                operator='elseif',
                arguments={'condition': '.....'},
                dependencies={...:''})

<hr style="height:1px;border-top:1px solid #0000FF" />

If the second condition is ```true```, create an empty container named ```workflow``` and then a new datacube with random data and dimensions.

In [ ]:
t4 = e1.newTask(name="Create container",
                type="ophidia",
                operator='oph_createcontainer',
                on_error='skip',
                arguments={'container': 'random',
                           'dim': 'lat|lon|time',
                           'dim_type': 'double|double|double',
                           'hierarchy': 'oph_base|oph_base|oph_time'},
                dependencies={...:''})
t5 = e1.newTask(name="Randcube",
                type="ophidia",
                operator='oph_randcube',
                arguments={'container': 'random',
                           'measure_type': 'double',
                           'exp_ndim': '2',
                           'dim': 'lat|lon|time',
                           'concept_level': 'c|c|d',
                           'dim_size': '16|10|10',
                           'measure': 'test_var',
                           'nhost': '1',
                           'nfrag': '16',
                           'ntuple': '10',
                           'hierarchy': 'oph_base|oph_base|oph_time'
                           'description': 'random cube'},
                dependencies={...:''})

<hr style="height:1px;border-top:1px solid #0000FF" />

Start the last sub-block of the selection block ```if```.

In [ ]:
t6 = e1.newTask(name="ELSE block",
                type="control",
                operator='else',
                arguments={},
                dependencies={...:''})

<hr style="height:1px;border-top:1px solid #0000FF" />

If neither of the two conditions is satisfied, first import a NetCDF file, then perform a subset operation over the datacube dimensions.

In [ ]:
t7 = e1.newTask(name="Import data",
                type="ophidia",
                operator='oph_importnc',
                arguments={'measure': '.....', 
                           'import_metadata': 'yes',
                           'imp_dim': 'time', 
                           'imp_concept_level': 'd',
                           'hierarchy': 'oph_base|oph_base|oph_time',
                           'description': 'Max Temp imported',
                           'input': '.....'},
                dependencies={...:''})
t8 = e1.newTask(name="Subset data",
                type="ophidia",
                operator='oph_subset', 
                arguments={'subset_filter': '.....', 
                           'subset_dims': '.....', 
                           'subset_type': 'coord', 
                           'description': 'Max Temp subsetted'},
                dependencies={t7:'.....'})

<hr style="height:1px;border-top:1px solid #0000FF" />

As last task, close the selection block ```if``` adding the proper dependencies.

In [ ]:
t9 = e1.newTask(name="ENDIF block",
                type="control",
                operator='endif',
                arguments={},
                dependencies={...:'', ...:'', ...:''})

<hr style="height:1px;border-top:1px solid #0000FF" />

We can check the workflow structure before submit it. 

In [ ]:
e1.check(visual=True)

<hr style="height:1px;border-top:1px solid #0000FF" />

And now, if everything is fine, we are ready to run the workflow with different input parameters to test the three possible branches!

**First case**: new datacube randomly generated inside a container named "workflow"
    
 <img src="../imgs/Selection_Interface_randcube.svg" alt="Selection_Interface_randcube" width="200">
 
**Second case**: datacube first imported, then subsetted

<img src="../imgs/Selection_Interface_import+subset.svg" alt="Selection_Interface_import+subset" width="200">

**Third case**: datacube imported and subsetted at the same time
    
 <img src="../imgs/Selection_Interface_importANDsubset.svg" alt="Selection_Interface_importANDsubset" width="200">

<hr style="height:1px;border-top:1px solid #0000FF" />

Define the workflow arguments:

In [ ]:
# Try all the possible options!
if_cond=...
elseif_cond=...

file="/home/ophidia/notebooks/tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc"
variable="tasmax"

# Try yourself with different filters!
lat_lon_time="-50:10|20:140|150:240" 

<hr style="height:1px;border-top:1px solid #0000FF" />

Run the workflow specifying the arguments in the proper order.

In [ ]:
w1 = Workflow(e1)
w1.submit(......, ......, ......, ......, ......)

<hr style="height:1px;border-top:1px solid #0000FF" />

Monitor the experiment execution graph

In [ ]:
w1.monitor(frequency=1, iterative=True, visual_mode=True)

<hr style="height:1px;border-top:1px solid #0000FF" />

Check your workspace

In [ ]:
cube.Cube.list(level=2)

<hr style="height:1px;border-top:1px solid #0000FF" />

Empty your workspace before running the workflow again

In [ ]:
cube.Cube.deletecontainer(container=".....",force='yes')
cube.Cube.deletecontainer(container="tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc",force='yes')

**NOTE:** You can check the completed exercise in the hands-on folder in the [**Selection_interface-filled.ipynb**](../Hands-on/3-Selection_interface-filled.ipynb).